In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/Madison/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [2]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()

# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
# Upon review of the exoplanet archive website the following variables appear to have impact
# on if a exoplanet is confirmed. Error were not taken into account as in most cases the 
# error was negligable compared to the value. koi_time0bk was removed as it is a time it 
# was observed not a feature of the object. 
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag']]
selected_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_impact,koi_impact_err1,koi_impact_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,0.586,0.059,-0.443,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,0.969,5.126,-0.077,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,1.276,0.115,-0.092,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,0.701,0.235,-0.478,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,0.762,0.139,-0.532,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
# removing variables with low variance (removing insignificant variables)
from sklearn.feature_selection import VarianceThreshold

var_model = VarianceThreshold(threshold=(.80 * (1 - .80)))
selected_data = var_model.fit_transform(selected_features)
len(selected_data[0])

31

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
y = df["koi_disposition"]
y_binary_encoded = pd.get_dummies(y)
# leaving as 3 column array for y since when it was dropped to 1 column for just confirmed
# the accuracy dropped greatly.

In [7]:
from sklearn.model_selection import train_test_split

#spliting training data, setting back to x and y for ease of modeling
X_train, X_test, y_train, y_test = train_test_split(selected_data, y_binary_encoded, random_state=57)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
# only X variable needs to be scaled since y is all binary encoding
# using training data to create the scaler 
X_MinMaxScaler = MinMaxScaler().fit(X_train)

In [9]:
X_train_scaled = X_MinMaxScaler.transform(X_train)
X_test_scaled = X_MinMaxScaler.transform(X_test)

# Train the Model



In [10]:
# first model tested is linear regression to get a base comparison 
from sklearn.linear_model import LinearRegression
model1 = LinearRegression()
model1.fit(X_train_scaled, y_train)

LinearRegression()

In [11]:
print(f"Training Data Score: {model1.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model1.score(X_test_scaled, y_test)}")

Training Data Score: 0.3790292586455745
Testing Data Score: 0.35832154821396944


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [12]:
# Create the GridSearchCV model
# using linear regression grid search params
from sklearn.model_selection import GridSearchCV
param_grid1 = {'fit_intercept': [True, False],
              'normalize': [True, False]}
grid1 = GridSearchCV(model1, param_grid1, verbose=3)

In [13]:
# Train the model with GridSearch
grid1.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] fit_intercept=True, normalize=True ..............................
[CV] . fit_intercept=True, normalize=True, score=-0.129, total=   0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] . fit_intercept=True, normalize=True, score=-0.232, total=   0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] . fit_intercept=True, normalize=True, score=-9.553, total=   0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] . fit_intercept=True, normalize=True, score=-1.369, total=   0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] .. fit_intercept=True, normalize=True, score=0.370, total=   0.0s
[CV] fit_intercept=True, normalize=False .............................
[CV] . fit_intercept=True, normalize=False, score=0.341, total=   0.0s
[CV] fit_intercept=True, normalize=False .............................
[CV] . fit_interc

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.2s finished


GridSearchCV(estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False],
                         'normalize': [True, False]},
             verbose=3)

In [14]:
print(grid1.best_params_)
print(grid1.best_score_)
#0.46564357827440184
#

{'fit_intercept': True, 'normalize': False}
0.21558095523924442


# Save the Model

In [15]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'mbfisher_model1.sav'
joblib.dump(grid1, filename)

['mbfisher_model1.sav']

In [16]:
predict_values = grid1.predict(X_test_scaled)
predict_values

array([[ 0.35095215,  0.57837677,  0.07066351],
       [ 0.44089508,  0.14931488,  0.40979904],
       [ 0.41370773,  0.33501816,  0.25128847],
       ...,
       [ 0.29619598,  0.00924301,  0.69457626],
       [-0.1818428 ,  0.10338211,  1.07846808],
       [ 0.12592316,  0.15591049,  0.71816725]])